<center>
    <h1>NBA Playoff Predictions</h1>
    <h3>Neel Shah</h3>
</center>

**Table of contents**<a id='toc0_'></a>    
1. [Introduction](#toc1_)    
2. [Data Collection](#toc2_)    
3. [Data Processing](#toc3_)    
4. [Exploratory Data Analysis & Visualization](#toc4_)    
5. [Modeling: Analysis, Hypothesis Testing, & Machine Learning](#toc5_)    
6. [Interpretation: Insight & Policy Decision](#toc6_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=true
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# 1. <a id='toc1_'></a>[Introduction](#toc0_)

The [National Basketball Association (NBA)](https://www.nba.com/) is a professional basketball league in the United States. There are [$30$ teams](https://www.nba.com/teams) in the league, divided evenly into $2$ conferences: the Eastern Conference and the Western Conference.

In the regular season, each team plays $82$ games. [NBA regular season standings](https://www.nba.com/standings) are determined by teams' win-loss records within their conferences.

The top $8$ teams from each conference advance to the playoffs. In the event of a tie in the standings, there is a [tie-breaking procedure](https://ak-static-int.nba.com/wp-content/uploads/sites/2/2017/06/NBA_Tiebreaker_Procedures.pdf) used to determine playoff seeding.

Starting in the $2019\text{-}20$ season, the NBA [added a play-in tournament](https://bleacherreport.com/articles/10031906-adam-silver-envisions-nba-play-in-tournament-becoming-a-fixture-in-this-league) to give the $9^\text{th}$ and $10^\text{th}$ place teams in each conference the opportunity to earn a spot in the playoffs. It works as follows:
- The $7^\text{th}$ and $8^\text{th}$ place teams play a game to determine the $7^\text{th}$ seed. The winner advances to the playoffs.
- The $9^\text{th}$ and $10^\text{th}$ place teams play an elimination game. The loser is eliminated.
- The loser of the $7/8$ game and the winner of the $9/10$ game play an elimination game to determine the $8^\text{th}$ seed. The winner advances to the playoffs; the loser is eliminated.

Once the final playoff seeding is determined, each team plays an opponent in a best-of-$7$ series. The first to win $4$ games advances to the next round. The first round is followed by the conference semifinals, then the conference finals, then the finals. The team that wins the NBA Finals in the NBA Champion.

The matchups for each round are determined using a traditional bracket structure, shown below:

![NBA Playoff Bracket](nba-playoff-bracket.png "NBA Playoff Bracket")

We want to perform some analysis to see if we can identify factors underlying teams' level of success in the playoffs. Our ultimate goal will be to predict the outcome of the NBA playoffs using data from the regular season.

Can we predict how likely a team is to:
- Win the championship?
- Make the finals?
- Make the conference finals?
- Make the conference semifinals?

These are some of the questions we want to answer as we go through the full data science pipeline.


# 2. <a id='toc2_'></a>[Data Collection](#toc0_)

# 3. <a id='toc3_'></a>[Data Processing](#toc0_)

# 4. <a id='toc4_'></a>[Exploratory Data Analysis & Visualization](#toc0_)

# 5. <a id='toc5_'></a>[Modeling: Analysis, Hypothesis Testing, & Machine Learning](#toc0_)

# 6. <a id='toc6_'></a>[Interpretation: Insight & Policy Decision](#toc0_)